In [1]:
import os
from tqdm import tqdm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.io import wavfile
import librosa
import librosa.display

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,Activation
from tensorflow.keras.optimizers import Adam

In [2]:
df=pd.read_csv("D:/MusicDataset/Metadata_Train.csv")
df2=pd.read_csv("D:/MusicDataset/Metadata_Test.csv")

In [3]:
df

,FileName,Class
0,1-E1-Major 00.wav,Sound_Guitar
1,1-E1-Major 01.wav,Sound_Guitar
2,1-E1-Major 02.wav,Sound_Guitar
3,1-E1-Major 03.wav,Sound_Guitar
4,1-E1-Major 04.wav,Sound_Guitar
...,...,...
2717,sawaar loon.wav,Sound_Vocal
2718,soona soona.wav,Sound_Vocal
2719,summane.wav,Sound_Vocal
2720,tangali.wav,Sound_Vocal


In [4]:
print(df['Class'].value_counts())
plt.figure(figsize=(22,10))
sns.countplot(df['Class'])
plt.xticks(rotation=0)

plt.show()

Class
Sound_Guitar    700
Sound_Drum      700
Sound_Violin    700
Sound_Piano     529
Sound_Vocal      93
Name: count, dtype: int64


ValueError: could not convert string to float: 'Sound_Guitar'

<Figure size 2200x1000 with 0 Axes>

In [5]:
df.drop(df[df['Class'].isin(['Sound_Violin'])].index, inplace=True)

In [6]:
df

,FileName,Class
0,1-E1-Major 00.wav,Sound_Guitar
1,1-E1-Major 01.wav,Sound_Guitar
2,1-E1-Major 02.wav,Sound_Guitar
3,1-E1-Major 03.wav,Sound_Guitar
4,1-E1-Major 04.wav,Sound_Guitar
...,...,...
2717,sawaar loon.wav,Sound_Vocal
2718,soona soona.wav,Sound_Vocal
2719,summane.wav,Sound_Vocal
2720,tangali.wav,Sound_Vocal


In [7]:
df2.drop(df2[df2['Class'].isin(['Sound_Violin'])].index, inplace=True)

In [8]:
df2

,FileName,Class
0,acoustic-guitar-logo-13084.wav,Sound_Guiatr
1,guitar-chords-70663.wav,Sound_Guiatr
2,guitar-intro-110935.wav,Sound_Guiatr
3,guitar-solo-27194.wav,Sound_Guiatr
4,guitar-solo-5999.wav,Sound_Guiatr
...,...,...
93,tere naina.wav,Sound_Vocal
94,tu hi meri shab.wav,Sound_Vocal
95,tu hi re.wav,Sound_Vocal
96,tujhe kitna.wav,Sound_Vocal


In [9]:
def Feature_extractor(file):
    
    audio,sample_rate=librosa.load(file_name,res_type='kaiser_fast')
    mfccs_features=librosa.feature.mfcc(y=audio,sr=sample_rate,n_mfcc=40)
    mfccs_scaled_features =np.mean(mfccs_features.T,axis=0)
    return mfccs_scaled_features

In [10]:
import os
from tqdm import tqdm

# Assuming df contains the DataFrame with columns "FileName" and "Class"
extracted_features = []

for index_num, row in tqdm(df.iterrows()):
    file_name = os.path.abspath(os.path.join("D:/MusicDataset/Train_submission/Train_submission", str(row["FileName"])))
    final_class_label = row["Class"]
    data = Feature_extractor(file_name)
    extracted_features.append([data, final_class_label])


2022it [07:44,  4.36it/s]


In [11]:
extracted_features_df=pd.DataFrame(extracted_features,columns=['features','class'])
extracted_features_df.head(20)

,features,class
0,"[-388.4289, 205.19734, 7.863611, 39.286667, 13...",Sound_Guitar
1,"[-359.54538, 171.62688, 10.192535, 37.61143, 1...",Sound_Guitar
2,"[-347.31915, 178.11375, 2.1611729, 38.960228, ...",Sound_Guitar
3,"[-389.92075, 177.8204, 10.302906, 42.16702, 14...",Sound_Guitar
4,"[-372.23615, 161.71027, 10.579021, 45.089127, ...",Sound_Guitar
5,"[-391.01813, 173.12811, 18.012999, 37.28042, 1...",Sound_Guitar
6,"[-386.7482, 162.40793, 15.373853, 42.025448, 2...",Sound_Guitar
7,"[-408.49643, 160.14922, 20.34738, 41.80766, 18...",Sound_Guitar
8,"[-398.09885, 147.52005, 24.257866, 43.7894, 3....",Sound_Guitar
9,"[-411.7316, 151.5299, 22.383932, 37.782017, 2....",Sound_Guitar


In [12]:
extracted_features_df

,features,class
0,"[-388.4289, 205.19734, 7.863611, 39.286667, 13...",Sound_Guitar
1,"[-359.54538, 171.62688, 10.192535, 37.61143, 1...",Sound_Guitar
2,"[-347.31915, 178.11375, 2.1611729, 38.960228, ...",Sound_Guitar
3,"[-389.92075, 177.8204, 10.302906, 42.16702, 14...",Sound_Guitar
4,"[-372.23615, 161.71027, 10.579021, 45.089127, ...",Sound_Guitar
...,...,...
2017,"[-484.7601, 104.058525, 15.061387, 24.160992, ...",Sound_Vocal
2018,"[-395.24832, 105.78999, -4.152595, 23.851385, ...",Sound_Vocal
2019,"[-384.6769, 114.452095, 0.62603873, 20.970564,...",Sound_Vocal
2020,"[-432.14066, 88.12449, 2.1225023, -1.633527, -...",Sound_Vocal


In [13]:
X=np.array(extracted_features_df['features'].tolist())
y=np.array(extracted_features_df["class"].tolist())

In [41]:
print(y)

[[0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 ...
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]]


In [14]:
X.shape


(2022, 40)

In [15]:
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
LE=LabelEncoder()
y=to_categorical(LE.fit_transform(y))

In [16]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [17]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [18]:
num_labels=y.shape[1]



In [19]:
num_labels

4

In [20]:
model=Sequential()
model.add(Dense(100,input_shape=(40,)))
model.add(Activation('relu'))
model.add(Dropout(0.3))

model.add(Dense(200))
model.add(Activation('relu'))
model.add(Dropout(0.3))


model.add(Dense(200))
model.add(Activation('relu'))
model.add(Dropout(0.3))

model.add(Dense(100))
model.add(Activation('relu'))
model.add(Dropout(0.3))

model.add(Dense(num_labels))
model.add(Activation('softmax'))

In [21]:
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 100)               4100      
                                                                 
 activation (Activation)     (None, 100)               0         
                                                                 
 dropout (Dropout)           (None, 100)               0         
                                                                 
 dense_1 (Dense)             (None, 200)               20200     
                                                                 
 activation_1 (Activation)   (None, 200)               0         
                                                                 
 dropout_1 (Dropout)         (None, 200)               0         
                                                                 
 dense_2 (Dense)             (None, 200)               4

In [22]:
model.compile(loss='categorical_crossentropy',metrics=['accuracy'],optimizer='adam')


In [23]:
from tensorflow.keras.callbacks import ModelCheckpoint
# tf.keras.callbacks.ModelCheckpoint
from datetime import datetime

num_epochs=100
num_batch_size=32

checkpointer=ModelCheckpoint(filepath='audio_classifier_model/audio_classification.hdf5',verbose=1,save_best_only=True)
start=datetime.now()
history = model.fit(X_train,y_train,batch_size=num_batch_size,epochs=num_epochs,validation_data=(X_test,y_test),callbacks=checkpointer)

duration=datetime.now()-start
print("Training Completed in time: ",duration)

Epoch 1/100


34/51 [===================>..........] - ETA: 0s - loss: 14.1108 - accuracy: 0.3686 
Epoch 1: val_loss improved from inf to 1.05208, saving model to audio_classifier_model\audio_classification.hdf5
51/51 [==============================] - 2s 9ms/step - loss: 10.9553 - accuracy: 0.3958 - val_loss: 1.0521 - val_accuracy: 0.6469
Epoch 2/100
45/51 [=========================>....] - ETA: 0s - loss: 3.1496 - accuracy: 0.4479

C:\Users\Sumanth\AppData\Roaming\Python\Python311\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(



Epoch 2: val_loss improved from 1.05208 to 0.96901, saving model to audio_classifier_model\audio_classification.hdf5
51/51 [==============================] - 0s 4ms/step - loss: 3.0684 - accuracy: 0.4447 - val_loss: 0.9690 - val_accuracy: 0.5556
Epoch 3/100
46/51 [==========================>...] - ETA: 0s - loss: 1.7612 - accuracy: 0.4925
Epoch 3: val_loss did not improve from 0.96901
51/51 [==============================] - 0s 3ms/step - loss: 1.7377 - accuracy: 0.4935 - val_loss: 0.9738 - val_accuracy: 0.5457
Epoch 4/100
50/51 [============================>.] - ETA: 0s - loss: 1.3702 - accuracy: 0.5344
Epoch 4: val_loss improved from 0.96901 to 0.95518, saving model to audio_classifier_model\audio_classification.hdf5
51/51 [==============================] - 0s 4ms/step - loss: 1.3707 - accuracy: 0.5325 - val_loss: 0.9552 - val_accuracy: 0.5926
Epoch 5/100
44/51 [========================>.....] - ETA: 0s - loss: 1.1447 - accuracy: 0.5561
Epoch 5: val_loss improved from 0.95518 to 0.9

In [24]:
from tensorflow.keras.models import load_model
filename="4de6f231e450d27dcf04591842f0f834a7802d27590bdaf34521dd9e.wav"
audio,sample_rate=librosa.load(filename,res_type='kaiser_fast')
mfccs_features=librosa.feature.mfcc(y=audio,sr=sample_rate,n_mfcc=40)
mfccs_scaled_features =np.mean(mfccs_features.T,axis=0)
from sklearn.preprocessing import LabelEncoder
model=load_model('audio_classifier_model/audio_classification.hdf5')
instrument_classes = ["Piano", "Guitar", "Drums", "Vocals"]
LE = LabelEncoder().fit(instrument_classes)
print(mfccs_scaled_features)
mfccs_scaled_features=mfccs_scaled_features.reshape(1,-1)
print(mfccs_scaled_features)

predicted_label = np.argmax(model.predict(mfccs_scaled_features), axis=-1)
print(predicted_label)

prediction_class=LE.inverse_transform(predicted_label)
print(prediction_class)

[-1.97125259e+02  1.31904236e+02 -1.49088402e+01  3.16940594e+01
 -1.01214485e+01  1.75017147e+01 -8.80626869e+00  2.13343978e+00
 -8.90354633e+00 -9.30018997e+00 -5.62300968e+00 -1.43834457e+01
 -1.07442446e+01 -1.93670235e+01 -1.13804512e+01 -6.68119049e+00
 -4.06562948e+00 -2.46168251e+01 -1.20662594e+01  6.49884194e-02
  1.09240091e+00  2.43879557e-01 -4.13289070e+00 -8.47594833e+00
 -2.23178768e+01 -1.13064551e+00  9.86099052e+00  1.02230701e+01
 -1.92450392e+00 -3.13192391e+00 -2.61982083e-01 -7.27817249e+00
  5.67436397e-01  2.14768028e+00  3.96985030e+00 -2.28359914e+00
 -3.19437838e+00 -2.21559620e+00  1.71233237e+00 -5.06967402e+00]
[[-1.97125259e+02  1.31904236e+02 -1.49088402e+01  3.16940594e+01
  -1.01214485e+01  1.75017147e+01 -8.80626869e+00  2.13343978e+00
  -8.90354633e+00 -9.30018997e+00 -5.62300968e+00 -1.43834457e+01
  -1.07442446e+01 -1.93670235e+01 -1.13804512e+01 -6.68119049e+00
  -4.06562948e+00 -2.46168251e+01 -1.20662594e+01  6.49884194e-02
   1.09240091e+00  

In [ ]:
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
LE=LabelEncoder()
y=to_categorical(LE.fit_transform(y))